In [26]:
from zipfile import ZipFile
import pandas as pd
import os
  
# Definindo o nome do arquivo de análise e seu caminho.
file_name = "Bilhetes.zip"
path = "C:/Users/danie/Desktop/Estudo/Eletivas/Big Data para dados públicos/Projeto-Big-Data/"
    
# Abrindo a arquivo zip com as informações de todos os meses
with ZipFile(os.path.join(path+file_name), 'r') as zip:
    # Printando o nome de todos os arquivo dentro do zip
    zip.printdir()  
    # Extraindo os arquivos para o diretório de trabalho
    zip.extractall()

File Name                                             Modified             Size
venda_passagem_01_2022.csv                     2022-10-22 16:20:22     17167260
venda_passagem_02_2022.csv                     2022-10-22 16:19:24     16620848
venda_passagem_03_2022.csv                     2022-10-22 16:18:58     17019407
venda_passagem_04_2022.csv                     2022-10-22 16:18:12      8500532
venda_passagem_05_2022.csv                     2022-10-22 16:17:38     16775582
venda_passagem_06_2022.csv                     2022-10-22 16:15:48     16292509
venda_passagem_07_2022.csv                     2022-10-22 16:00:36     19957908
venda_passagem_09_2021.csv                     2022-10-22 16:22:58     15840935
venda_passagem_09_2022.csv                     2022-10-22 15:54:58     17361183
venda_passagem_10_2021.csv                     2022-10-22 16:22:22     16216048
venda_passagem_11_2021.csv                     2022-10-22 16:21:48     16376100
venda_passagem_12_2021.csv              

In [27]:
# Criando o Data Frame de análise em branco e depois fazendo a inserção das informações de todos
# os meses
bilhetes = pd.DataFrame()
for file in zip.namelist():
    bilheteMes = pd.read_csv(file, sep=";", encoding="latin-1")
    try:
        bilhetes = pd.concat([bilhetes, bilheteMes], ignore_index=True)
        # Mostrando a evolução do tamanho da tabela 
        print(f"O arquivo de análise contém: '{len(bilhetes)}' linhas!") 
    except error as Exception:
        print(error)

O arquivo de análise contém: '132157' linhas!
O arquivo de análise contém: '260331' linhas!
O arquivo de análise contém: '391086' linhas!
O arquivo de análise contém: '457984' linhas!
O arquivo de análise contém: '586954' linhas!
O arquivo de análise contém: '713771' linhas!
O arquivo de análise contém: '867262' linhas!
O arquivo de análise contém: '988633' linhas!
O arquivo de análise contém: '1124196' linhas!
O arquivo de análise contém: '1248443' linhas!
O arquivo de análise contém: '1374418' linhas!
O arquivo de análise contém: '1527032' linhas!


In [39]:
# Limpando linhas com valores NaN
if len(bilhetes[bilhetes.isna().any(axis=1)])!=0:
    print(f"O arquivo contém '{len(bilhetes[bilhetes.isna().any(axis=1)])}' linhas com NaN que"+
          "devem ser apagadas")
    bilhetes.dropna(inplace=True)
print(f"O arquivo contém '{len(bilhetes[bilhetes.isna().any(axis=1)])}' linhas com NaN!")

O arquivo contém '0' linhas com NaN!


In [41]:
bilhetes.head()

,mes_emissao_bilhete,mes_viagem,ponto_origem_viagem,ponto_destino_viagem,tipo_servico,tipo_gratuidade,media_valor_total,dp_valor_total,quantidade_bilhetes
0,01/2022,01/2022,BELEM/PA,MARACACUME/MA,Convencional com sanitário,Tarifa Normal - sem desconto,80.47,6.54,16
1,01/2022,02/2022,CAJAZEIRAS/PB,JUAZEIRO DO NORTE/CE,Convencional com sanitário,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",44.16,0.37,6
2,01/2022,01/2022,CUIABA/MT,NAVIRAI/MS,Executivo,"Gratuidade de Criança - Inciso XVII, art. 29 d...",0.00,0.00,3
3,01/2022,01/2022,TIMOTEO/MG,SAO PAULO/SP,Convencional com sanitário,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",0.00,0.00,5
4,01/2022,01/2022,MOJI-MIRIM/SP,POUSO ALEGRE/MG,Executivo,Tarifa Normal - sem desconto,39.36,1.74,11


In [42]:
bilhetes.tail()

,mes_emissao_bilhete,mes_viagem,ponto_origem_viagem,ponto_destino_viagem,tipo_servico,tipo_gratuidade,media_valor_total,dp_valor_total,quantidade_bilhetes
1527027,12/2021,12/2021,INDAIATUBA/SP,CASCAVEL/PR,Leito com ar condicionado,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",305.69,0.00,1
1527028,12/2021,12/2021,RIO VERDE DE MATO GROSSO/MS,SARANDI/RS,Executivo,Tarifa Normal - sem desconto,375.23,0.00,2
1527029,12/2021,12/2021,MARACACUME/MA,CAXIAS/MA,Executivo,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",78.00,0.00,7
1527030,12/2021,12/2021,ARARAQUARA/SP,CORNELIO PROCOPIO/PR,Leito com ar condicionado,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",182.06,0.00,1
1527031,12/2021,12/2021,ESTREITO/MA,BELEM/PA,Executivo,Tarifa Normal - sem desconto,156.24,16.86,4
